# 1 Initialization

In [1]:
#from fastai.imports import *
#from fastai.structured import *
import os
import math
import pandas as pd
from pandas.api.types import is_string_dtype, is_numeric_dtype
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from IPython.display import display
import re
from sklearn import metrics
import numpy as np

C:\Users\Aaron\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [4]:
PATH = "../2/data/groceries/"
!ls {PATH}

holidays_events.csv
items.csv
oil.csv
sample_submission.csv
test.csv
train.csv
transactions.csv


# 2 Reading Data
## to ensure a header is assigned the correct datatypes, define the dictionary you want it to be mapped to.

In [5]:
types = {'id': 'int64',
        'item_nbr': 'int32',
        'store_nbr': 'int8',
        'unit_sales': 'float32',
        'onpromotion': 'object'}

In [6]:
%%time
df_all = pd.read_csv(f'{PATH}train.csv', parse_dates = ['date'], dtype = types,
                    infer_datetime_format = True)

Wall time: 5min 11s


### in practice, you don't need to read the whole thing in. use the linux command 'shuf'. This will parse a sample and the sample is what you use to work with instead of waiting for the entire data to come in.

## Now we will fill in missing values with false, since we know that there is na in the 'onpromotion' section

In [7]:
df_all['onpromotion'].fillna(False, inplace = True)# changes all NaN entries to False, and keeps anything that is an entry.
df_all['onpromotion'] = df_all['onpromotion'].map({'False' : False, 'True': True})#maps the entry 'False' to boolean False
df_all['onpromotion'] = df_all['onpromotion'].astype(bool)#changes the dtype to boolean for the entire column

In [9]:
%time df_all.to_feather('tmp/raw_groceries')

Wall time: 8.81 s


In [10]:
%time df_all.describe(include = 'all')# the param 'include' allows describing EVERYTHING. Useful for stats items

Wall time: 28.5 s


,id,date,store_nbr,item_nbr,unit_sales,onpromotion
count,1.254970e+08,125497040,1.254970e+08,1.254970e+08,1.254970e+08,125497040
unique,NaN,1684,NaN,NaN,NaN,2
top,NaN,2017-07-01 00:00:00,NaN,NaN,NaN,False
freq,NaN,118194,NaN,NaN,NaN,96028767
first,NaN,2013-01-01 00:00:00,NaN,NaN,NaN,NaN
last,NaN,2017-08-15 00:00:00,NaN,NaN,NaN,NaN
mean,6.274852e+07,NaN,2.746458e+01,9.727692e+05,5.319669e+00,NaN
std,3.622788e+07,NaN,1.633051e+01,5.205336e+05,2.306714e+01,NaN
min,0.000000e+00,NaN,1.000000e+00,9.699500e+04,-1.537200e+04,NaN
25%,3.137426e+07,NaN,1.200000e+01,5.223830e+05,2.000000e+00,NaN


In [12]:
df_test = pd.read_csv(f'{PATH}test.csv', parse_dates = ['date'], dtype = types,
                     infer_datetime_format = True)
df_test['onpromotion'].fillna(False, inplace = True)
df_test['onpromotion'] = df_test['onpromotion'].map({'False': False, 'True': True})
df_test['onpromotion'] = df_test['onpromotion'].astype(bool)
df_test.describe(include = 'all')

,id,date,store_nbr,item_nbr,onpromotion
count,3.370464e+06,3370464,3.370464e+06,3.370464e+06,3370464
unique,NaN,16,NaN,NaN,2
top,NaN,2017-08-27 00:00:00,NaN,NaN,False
freq,NaN,210654,NaN,NaN,3171867
first,NaN,2017-08-16 00:00:00,NaN,NaN,NaN
last,NaN,2017-08-31 00:00:00,NaN,NaN,NaN
mean,1.271823e+08,NaN,2.750000e+01,1.244798e+06,NaN
std,9.729693e+05,NaN,1.558579e+01,5.898362e+05,NaN
min,1.254970e+08,NaN,1.000000e+00,9.699500e+04,NaN
25%,1.263397e+08,NaN,1.400000e+01,8.053210e+05,NaN


## Now that the data has been analyzed, we understand we have 4 years of training data to test on 2 weeks of data.
### Now what to infer from this?
### We should grab the most recent since data from 2013 is not as relevant. That doesn't mean it's not relevant, but it will be weighted much less. For the base case, we should focus only on the most recent,then narrow down to the most important peices

In [13]:
df_all.tail()

,id,date,store_nbr,item_nbr,unit_sales,onpromotion
125497035,125497035,2017-08-15,54,2089339,4.0,False
125497036,125497036,2017-08-15,54,2106464,1.0,True
125497037,125497037,2017-08-15,54,2110456,192.0,False
125497038,125497038,2017-08-15,54,2113914,198.0,True
125497039,125497039,2017-08-15,54,2116416,2.0,False


In [ ]:
df_all = pd.read_feather('tmp/raw_groceries')

In [15]:
df_all['unit_sales'] = np.log1p(np.clip(df_all['unit_sales'], 0, None))#the evaluation is RMSL+1E, since log(0) is not real.

### The competition says to count negatives in unit sales as 0, which is what np.cllip allows us to do, we set the minimum as 0, and put None as the maximum since there is no ceiling.